In [11]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [12]:
# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 500000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
attn_dropout = 0.0

In [13]:
torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('./input/tinyshakespeare.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]


In [14]:
# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
@torch.no_grad()
def estimate_loss(model):
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean().item()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size, attn_dropout=0.0):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(attn_dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size, attn_dropout=0.0):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size, attn_dropout=attn_dropout) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(attn_dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd, attn_dropout=0.0):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(attn_dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head, attn_dropout=0.0):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size, attn_dropout=attn_dropout)
        self.ffwd = FeedFoward(n_embd, attn_dropout=attn_dropout)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)


    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self, attn_dropout=0.0):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head, attn_dropout=attn_dropout) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')


0.209729 M parameters


In [15]:
import json
import os
import time

attn_dropout_values = [0.0, 0.2, 0.5]

for attn_dropout in attn_dropout_values:
    print(f"Training model with attn_dropout={attn_dropout}")

    start_time = time.time()
    
    model = BigramLanguageModel(attn_dropout=attn_dropout).to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
    
    train_losses = []
    val_losses = []

    for iter in range(max_iters):
        if iter % eval_interval == 0 or iter == max_iters - 1:
            losses = estimate_loss(model)
            train_losses.append(losses['train'])
            val_losses.append(losses['val'])
            print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
        
        xb, yb = get_batch('train')
        
        logits, loss = model(xb, yb)
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()

    end_time = time.time()
    training_time = end_time - start_time


    folder_path = "./loss_time/attn" 
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    loss_file_path = os.path.join(folder_path, f'attn_dropout_{attn_dropout}_losses.json')
    with open(loss_file_path, 'w') as f:
        json.dump({"train_losses": train_losses, "val_losses": val_losses}, f)
    
    time_file_path = os.path.join(folder_path, f'attn_dropout_{attn_dropout}_training_time.json')
    with open(time_file_path, 'w') as f:
        json.dump({"training_time": training_time}, f)
    
    print(f"Results for attn_dropout={attn_dropout} saved.")


Training model with attn_dropout=0.0
step 0: train loss 4.3823, val loss 4.3857
step 100: train loss 2.6707, val loss 2.6817
step 200: train loss 2.5247, val loss 2.5219
step 300: train loss 2.4221, val loss 2.4415
step 400: train loss 2.3475, val loss 2.3505
step 500: train loss 2.2924, val loss 2.3260
step 600: train loss 2.2416, val loss 2.2599
step 700: train loss 2.2065, val loss 2.2239
step 800: train loss 2.1704, val loss 2.2044
step 900: train loss 2.1150, val loss 2.1560
step 1000: train loss 2.0894, val loss 2.1337
step 1100: train loss 2.0630, val loss 2.1217
step 1200: train loss 2.0401, val loss 2.1068
step 1300: train loss 2.0056, val loss 2.0608
step 1400: train loss 1.9919, val loss 2.0514
step 1500: train loss 1.9745, val loss 2.0417
step 1600: train loss 1.9417, val loss 2.0173
step 1700: train loss 1.9377, val loss 2.0090
step 1800: train loss 1.9119, val loss 1.9975
step 1900: train loss 1.8968, val loss 1.9909
step 2000: train loss 1.8816, val loss 1.9809
step 2100

In [1]:
import json
import matplotlib.pyplot as plt
import os

# Specify the folder path where the loss JSON files are stored
folder_path = "./loss_time/attn"
# Specify the path to save the visualization result
visualization_save_path = "./visualization/attn_loss_curves.png"

# Use a pattern or a specific part of the file name to identify loss files
# Assuming loss files contain '_losses' in their filenames
loss_files = [f for f in os.listdir(folder_path) if f.endswith('_losses.json')]

# Visualize the loss curves for each dropout configuration
plt.figure(figsize=(10, 6))

for file in loss_files:
    file_path = os.path.join(folder_path, file)
    
    # Read the loss values from the JSON file
    with open(file_path, 'r') as f:
        losses = json.load(f)
    
    # Plot the validation loss curves
    # Extracting the dropout value from the filename for labeling
    dropout_label = file.split('_')[2]  # Assumes format 'attn_dropout_{value}_losses.json'
    plt.plot(losses['val_losses'], label=f"Validation Loss Dropout {dropout_label}")

plt.title("Loss Curves for Different Attention Dropout Configurations")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.grid(True)

# Save the image
plt.savefig(visualization_save_path)
plt.close()

print(f"Visualization saved to {visualization_save_path}")

Visualization saved to ./visualization/attn_loss_curves.png
